<a href="https://colab.research.google.com/github/pedrohcaminha/ProjetoPLN/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 baixando os dados

In [14]:
! pip install kaggle

In [15]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [16]:
! cp kaggle.json ~/.kaggle/

In [17]:
! chmod 600 ~/.kaggle/kaggle.json

In [18]:
! kaggle datasets download hassanamin/atis-airlinetravelinformationsystem

  0% 0.00/139k [00:00<?, ?B/s]
100% 139k/139k [00:00<00:00, 46.6MB/s]


In [19]:
! unzip atis-airlinetravelinformationsystem.zip

Archive:  atis-airlinetravelinformationsystem.zip
  inflating: atis_intents.csv        
  inflating: atis_intents_test.csv   
  inflating: atis_intents_train.csv  


In [20]:
! ls

atis-airlinetravelinformationsystem.zip  atis_intents_test.csv	 kaggle.json
atis_intents.csv			 atis_intents_train.csv  sample_data


# 2 lendo e separando os dados

importando bibliotecas

In [21]:
import spacy
import csv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import spatial
import nltk
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers.embeddings import Embedding
from keras.layers import Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Input, Dense
from keras.models import Sequential

from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


dados de treino e teste

In [22]:
train = pd.read_csv("atis_intents_train.csv", header=None)
test = pd.read_csv("atis_intents_test.csv", header=None)

# 3 pre-processamento

stop words

In [23]:
from nltk.corpus import stopwords
words = set(stopwords.words("english"))
print(words)

{'how', 'hasn', 'y', 'only', 'just', 'from', 'haven', 'some', "doesn't", 'my', 'our', 'an', 't', 'shouldn', 'she', "that'll", 'against', "you'd", 'whom', 've', 'below', 'not', 'down', 'himself', 'his', 'shan', 'with', "it's", 'a', 'more', 'her', 'him', 'yours', 'until', 'at', 'again', 'than', 'doesn', 'have', 'where', 'herself', 'or', 'so', "shouldn't", "mightn't", 'its', 'why', 'both', 'which', 'who', 'these', "wasn't", 'too', 'd', 'should', 'yourself', 'are', 'their', 'i', 'own', 'do', 'been', 'was', 'am', 'for', 'needn', 'wouldn', 'over', 'few', 'nor', 'can', 'having', 'through', 'itself', 'you', 'other', "won't", 'after', 'any', 'to', 'were', 'into', 'while', 'off', 'before', "mustn't", 'hers', 'once', 'me', 'aren', 'o', 'he', "aren't", 'because', 'that', 'ma', 'don', 'couldn', 's', 'on', "you've", "didn't", "needn't", 'did', 'all', 'we', "shan't", 'won', 'during', 'had', 'mightn', 're', 'between', 'no', "couldn't", 'those', 'doing', 'further', 'each', 'most', 'very', 'weren', 'had

In [24]:
train['text'] = train[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))
test['text'] = test[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))

train['text'] = train['text'].str.replace('\d+', '')
test['text'] = test['text'].str.replace('\d+', '')

In [25]:
text = train['text']
labels = train[0]
test_text = test['text']
test_labels = test[0]

In [26]:
train.head()

,0,1,text
0,atis_flight,i want to fly from boston at 838 am and arriv...,want fly boston arrive denver morning
1,atis_flight,what flights are available from pittsburgh to...,flights available pittsburgh baltimore thursda...
2,atis_flight_time,what is the arrival time in san francisco for...,arrival time san francisco flight leaving was...
3,atis_airfare,cheapest airfare from tacoma to orlando,cheapest airfare tacoma orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...,round trip fares pittsburgh philadelphia dollars


# 4 vetorizando palavras

In [27]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
tok.fit_on_texts(text)
word_index = tok.word_index

In [28]:
word_index['plane']

227

In [29]:
max_vocab_size = len(word_index) + 1
input_length = 25

In [30]:
train_data_tokens = tok.texts_to_sequences(text)
test_data_tokens = tok.texts_to_sequences(test_text)

In [31]:
train_input = pad_sequences(train_data_tokens, input_length)
test_input = pad_sequences(test_data_tokens, input_length)

# 5 codificando os rotulos

In [32]:
a = labels

label_transformer = preprocessing.LabelEncoder()
label_transformer.fit(labels)

LabelEncoder()

In [33]:
labels = label_transformer.transform(labels)
b = labels
test_labels = label_transformer.transform(test_labels)
c = labels

In [34]:
labels = to_categorical(np.asarray(labels))
d = labels
test_labels = to_categorical(np.asarray(test_labels))

In [35]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 6 separação de treino e teste (para treinamento)

In [36]:
X_train, X_val, y_train, y_val = train_test_split(train_input, labels, test_size=0.2, random_state=42)
X_train

array([[  0,   0,   0, ...,  52, 477, 139],
       [  0,   0,   0, ..., 105,  50,  64],
       [  0,   0,   0, ...,   5,   7,   3],
       ...,
       [  0,   0,   0, ...,   2,  10,   9],
       [  0,   0,   0, ..., 218, 137,  93],
       [  0,   0,   0, ...,  42,  46,   3]], dtype=int32)

# 7 deep learning

In [37]:
model = Sequential()
model.add(Embedding(max_vocab_size, 300, input_length=input_length, trainable=False))
model.add(Conv1D(filters=32, kernel_size=8, activation='selu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='selu'))
model.add(Dense(8, activation='sigmoid'))

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           189600    
_________________________________________________________________
conv1d (Conv1D)              (None, 18, 32)            76832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 9, 32)             0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                2890      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 88        
Total params: 269,410
Trainable params: 79,810
Non-trainable params: 189,600
_____________________________________________

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=6, verbose=1)

Epoch 1/6
121/121 [==============================] - 31s 12ms/step - loss: 1.0887 - accuracy: 0.7190
Epoch 2/6
121/121 [==============================] - 1s 12ms/step - loss: 0.6236 - accuracy: 0.8332
Epoch 3/6
121/121 [==============================] - 1s 11ms/step - loss: 0.4533 - accuracy: 0.8751
Epoch 4/6
121/121 [==============================] - 1s 12ms/step - loss: 0.3125 - accuracy: 0.9115
Epoch 5/6
121/121 [==============================] - 1s 11ms/step - loss: 0.2209 - accuracy: 0.9405
Epoch 6/6
121/121 [==============================] - 1s 12ms/step - loss: 0.1631 - accuracy: 0.9550


In [40]:
model.evaluate(X_val, y_val)

31/31 [==============================] - 1s 4ms/step - loss: 0.2220 - accuracy: 0.9369


[0.2220492660999298, 0.936918318271637]

In [41]:
predictions = model.predict(test_input)

In [42]:
def acc(y_true, y_pred):
    return np.equal(np.argmax(y_true, axis=-1), np.argmax(y_pred, axis=-1)).mean()

print(acc(test_labels, predictions))

0.93875


In [43]:
def get_intent(sentence):
  data = [[sentence]]
  df = pd.DataFrame(data)
  entrada = df[0].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))
  entrada = tok.texts_to_sequences(entrada)
  entrada = pad_sequences(entrada, input_length)
  prediction = model.predict(entrada)
  # return np.argmin(prediction)
  return a[np.where(b == np.argmax(prediction))[0][0]]

#  8 intencoes

In [44]:
get_intent("what kinds of planes are used by american airlines")

'atis_ground_service'

In [45]:
get_intent("show me the first class fares from boston to denver")

'atis_airfare'

In [46]:
get_intent("show me the flights from boston to denver")

'atis_flight'

# 9 generating responses

In [113]:
intents = {}
for i in range(8):
  intents.update({a[np.where(b == i)[0][0]]: []})

intents['atis_abbreviation'].append("it means @")
intents['atis_aircraft'].append("the aircraft used is @")
intents['atis_airfare'].append("the price is @")
intents['atis_airfare'].append("the price in @ can vary from @ to @")
intents['atis_airfare'].append("the price in @ is @")
intents['atis_airfare'].append("the price is @")
intents['atis_airline'].append("the airline is @")
intents['atis_flight'].append("the available flights are @")
intents['atis_flight_time'].append("the time is @")
intents['atis_ground_service'].append("the service will be @ available at @")
intents['atis_quantity'].append("there are at least @")

intents

{'atis_abbreviation': ['it means @'],
 'atis_aircraft': ['the aircraft used is @'],
 'atis_airfare': ['the price is @',
  'the price in @ can vary from @ to @',
  'the price in @ is @',
  'the price is @'],
 'atis_airline': ['the airline is @'],
 'atis_flight': ['the available flights are @'],
 'atis_flight_time': ['the time is @'],
 'atis_ground_service': ['the service will be @ available at @'],
 'atis_quantity': ['there are at least @']}

In [114]:
! pip install termcolor
from termcolor import colored


In [115]:
import random
def response(text):
  intent = get_intent(text)
  print(colored(intent, 'red'))
  print(colored("BOT: " + random.choice(intents[intent]), 'green'))

In [116]:
entrada = input("YOU: ")
response(entrada)

YOU: which aircraft types are used in NYC
atis_aircraft
BOT: the aircraft used is @


# 10 Entidades

In [117]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [118]:
import nltk.tokenize as nt

In [119]:
text="show me the flights from boston to denver"
ss=nt.sent_tokenize(text)
ss

['show me the flights from boston to denver']

In [120]:
tokenized_sent=[nt.word_tokenize(sent) for sent in ss]
tokenized_sent

[['show', 'me', 'the', 'flights', 'from', 'boston', 'to', 'denver']]

* CC  	coordinating conjunction
* CD 	 cardinal digit
* DT  	determiner
* EX 	existential there (like: "there is" ... think of * it like "there exists")
* FW   	foreign word
* IN   	preposition/subordinating conjunction
* JJ  	adjective    'big'
* JJR 	adjective, comparative    'bigger'
* JJS 	adjective, superlative    'biggest'
* LS 	List marker    1)
* MD 	modal    could, will
* NN 	noun, singular 'desk'
* NNS 	 noun plural    'desks'
* NNP 	proper noun, singular    'Harrison'
* NNPS 	 proper noun, plural    'Americans'
* PDT 	predeterminer    'all the kids'
* POS 	possessive ending    parent's
* PRP 	personal pronoun    I, he, she
* PRPdollar 	 possessive pronoun    my, his, hers
* RB 	adverb    very, silently,
* RBR 	adverb, comparative    better
* RBS 	Adverb, superlative    best
* RP 	particle    give up
* TO 	to go 'to' the store.
* UH 	 interjection    errrrrrrrm
* VB 	Verb, base form    take
* VBD 	verb, past tense, took
* VBG 	Verb, gerund/present participle    taking
* VBN 	verb, past participle taken
* VBP 	verb, sing. present, non-3d    take
* VBZ 	 verb, 3rd person sing. present    takes
* WDT 	wh-determiner  which
* WP 	 wh-pronoun    who, what
* WPdollar 	possessive wh-pronoun    whose
* WRB 	wh-adverb    where, when

In [121]:
nltk.pos_tag(['boston'])

[('boston', 'NN')]

In [122]:
pos_sentences=[nltk.pos_tag(sent) for sent in tokenized_sent]
pos_sentences

[[('show', 'VB'),
  ('me', 'PRP'),
  ('the', 'DT'),
  ('flights', 'NNS'),
  ('from', 'IN'),
  ('boston', 'NN'),
  ('to', 'TO'),
  ('denver', 'VB')]]

In [123]:
def extract_NN(sent):
    grammar = r"""
    NBAR:
        # Nouns and Adjectives, terminated with Nouns
        {<NN.*>*<NN.*>}

    NP:
        {<NBAR>}
        # Above, connected with in/of/etc...
        {<NBAR><IN><NBAR>}
    """
    chunker = nltk.RegexpParser(grammar)
    ne = set()
    chunk = chunker.parse(nltk.pos_tag(nltk.word_tokenize(sent)))
    for tree in chunk.subtrees(filter=lambda t: t.label() == 'NP'):
        ne.add(' '.join([child[0] for child in tree.leaves()]))
    return ne

In [124]:
extract_NN("show me the flights from boston to denver")

{'boston', 'flights'}

# 11 generating responses with entities

In [125]:
data = [['atis_aircraft', 'NYC', 'AC-202'], ['atis_aircraft', 'boston', 'Airbus B212']]

In [126]:
def get_data(text):
  intent = get_intent(text)
  options = []
  for d in data:
    if d[0] == intent:
      options.append(d)
  entities = extract_NN(text)
  answer = "no data found"
  for d in options:
    if d[1] in entities:
      answer = d[2]
  return answer

In [127]:
import random
def response(text):
  intent = get_intent(text)
  print(colored(intent, 'red'))
  print(colored(extract_NN(text), 'blue'))
  answer = random.choice(intents[intent])
  print(colored("BOT: " + answer.replace("@", get_data(text)), 'green'))

In [128]:
entrada = input("YOU: ")
response(entrada)

YOU: which aircraft types are used in NYC
atis_aircraft
{'aircraft types', 'NYC'}
BOT: the aircraft used is AC-202


In [129]:
entrada = input("YOU: ")
response(entrada)

YOU: which aircraft types are used in boston
atis_aircraft
{'aircraft types', 'boston'}
BOT: the aircraft used is Airbus B212
